# Visualizing PSD, SCV and Specified Frequency distributions

**Objective:**
visualizing global effects through PSD and SCV and local effects through specified frequency to introspect the electrophysiology data.

In [1]:
cd /Users/SP/code/spectralCV/results/kjm_digits/bp

/Users/SP/code/spectralCV/results/kjm_digits/bp


In [2]:
# locating local path for spectralCV
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/Users/SP/code/spectralCV')

In [3]:
%matplotlib inline

# imports
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import os

import neurodsp as ndsp
from scv_funcs import lfpca
from scv_funcs import utils

CKEYS = plt.rcParams['axes.prop_cycle'].by_key()['color']
font = {'family' : 'arial',
        'weight' : 'regular',
        'size'   : 15}

import matplotlib
matplotlib.rc('font', **font)

In [4]:
# bokeh imports
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from scipy.stats import expon
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import PreText, Select, Slider
from bokeh.io import curdoc
from bokeh.layouts import row, column
from bokeh.models import Span

# output to notebook
output_notebook()

# output to  html
# from bokeh.plotting import figure, output_file, save
# output_file("kjm_bp.html")

Loading BokehJS ...

In [5]:
# electrode location
elec_def = {
    '1': 'dorsal M1',
    '3': 'dorsal S1',
    '4': 'ventral S1+M1',
    '6': 'frontal (non-R)',
    '7': 'parietal (non-R)',
    '8': 'temporal',
    '9': 'occipital'
}
fs = 1000.

In [6]:
# obtain data information
trial_data = np.load('trial_info.npz')
trial_info = trial_data['trial_info']
elec_regions = trial_data['elec_regions']
conditions = ['pre', 'move', 'whole']
lfpca_all = []
for cond in conditions:
    lfpca_all.append(lfpca.lfpca_load_spec(cond+'.npz'))
    
lf_pre = lfpca_all[0]
lf_move = lfpca_all[1]
lf_whole = lfpca_all[2]

In [8]:
# updating histogram based on channel selection

# list of channel selection
# chan_sel = []
# for plot_chan in range(lf.numchan):
#     try:
#         ele_name = elec_def[elec_regions[plot_chan].astype(str)]
#     except:
#         ele_name = 'others'
#     chan_sel.append('%i: '%plot_chan+ele_name)

# update histogram based on frequency, channel, bins on slider
# def update_spct_hist(x, y, f=10, channel=0, numbins=20):
#     plot_chan = int(channel.split(':')[0])
#     plot_freq = np.where(lf.f_axis==f)[0][0]
#     y, x = np.histogram(lf.spg[plot_chan,plot_freq,:], bins=numbins, density=True)
#     # update histogram with data from frequency f
#     hist_plot.data_source.data['left'] = x[:-1]
#     hist_plot.data_source.data['right'] = x[1:]
#     hist_plot.data_source.data['top'] = y    
#     # update fitted
#     rv = expon(scale=sp.stats.expon.fit(lf.spg[plot_chan,plot_freq,:],floc=0)[1])
#     fit_plot.data_source.data['x'] = x
#     fit_plot.data_source.data['y'] = rv.pdf(x)
    
#     hist_fig.title.text = elec_def[elec_regions[plot_chan].astype(str)] + ': f=%.1fHz, p=%.4f '%(f,lf.ks_pvals[plot_chan,plot_freq])
#     push_notebook()

In [22]:
# grabbing the values for histogram, adding interaction
# y, x = np.histogram(lf.spg[0,10,:], bins=20, density=True)
# hist_fig = figure(plot_height=400, plot_width=400, x_axis_label='Power', y_axis_label='Probability',background_fill_color="#E8DDCB")
# hist_plot = hist_fig.quad(top=y, bottom=0, left=x[:-1], right=x[1:], fill_color="#036564", line_color="#033649")
# fit_plot = hist_fig.line(x[:-1],y, line_width=8,alpha=0.7,line_color="#D95B43",legend='Fit PDF')
# show(hist_fig, notebook_handle=True)
# interact(update_spct_hist, y=y, x=x, f=(1,100), numbins=range(10,55,5), channel=chan_sel)

In [21]:
# using lf_pre as an example
lf = lf_pre

# grabbing channel count from psd
chan_count, freq = lf.psd.shape
# mapping all the channels
DEFAULT_TICKERS = list(map(str, range(chan_count)))

# creating a selector and slider
ticker = Select(value='0', title='channel', options=DEFAULT_TICKERS)
freq_slider = Slider(start=1, end=100, value=10, step=1, title="Frequency")
bin_slider = Slider(start=10, end=55, value=20, step=5, title="Number of bins")

# initializing values for frequency, psd, scv plot
freq_vals = np.log(lf.f_axis[1:])
psd_vals = np.log(lf.psd[0].T[1:])
scv_vals = np.log(lf.scv[0].T[1:])
select_freq = np.log(10)

# create data and selection tools
source = ColumnDataSource(data=dict(freq_vals=freq_vals, psd_vals=psd_vals, scv_vals=scv_vals))
TOOLS = "box_select,lasso_select,help"

# setting up plots
psd_plot = figure(tools=TOOLS, plot_width=300, plot_height=300, title='PSD')
scv_plot = figure(tools=TOOLS, plot_width=300, plot_height=300, title='SCV')

# customize plot to psd
def create_psd_plot(psd_plot, source, freq):
    # create a new plot and add a renderer
    psd_plot.line('freq_vals', 'psd_vals', source=source, color='navy')
    psd_plot.circle('freq_vals', 'psd_vals', source=source, size=4, color='darkgrey', alpha=0.2)
    psd_plot.legend.location = 'top_left'
    psd_plot.xaxis.axis_label = 'Log Frequency (Hz)'
    psd_plot.yaxis.axis_label = 'Log Power/Frequency (dB/Hz)'
    psd_plot.grid.grid_line_alpha=0.3
    # adding vertical line that corresponds to the changing freq slider
    vline = Span(location=freq, dimension='height', line_color='red', line_width=3)
    psd_plot.renderers.extend([vline])
    
# customize plot to psd
def create_scv_plot(scv_plot, source, freq):
    # create another new plot and add a renderer
    scv_plot.line('freq_vals', 'scv_vals', source=source, color='navy')
    scv_plot.circle('freq_vals', 'scv_vals', source=source, size=4, color='darkgrey', alpha=0.2)
    scv_plot.legend.location='top_left'
    scv_plot.xaxis.axis_label = 'Log Frequency (Hz)'
    scv_plot.yaxis.axis_label = 'Log (Unitless)'
    scv_plot.grid.grid_line_alpha=0.3
    vline = Span(location=freq, dimension='height', line_color='red', line_width=3)
    scv_plot.renderers.extend([vline])

# customize histogram, plot fitted exponential dist
def create_hist(lf):
    y, x = np.histogram(lf.spg[0,10,:], bins=20, density=True)
    hist_fig = figure(plot_height=400, plot_width=400, x_axis_label='Power', y_axis_label='Probability',background_fill_color="#E8DDCB")
    hist_plot = hist_fig.quad(top=y, bottom=0, left=x[:-1], right=x[1:], fill_color="#036564", line_color="#033649")
    fit_plot = hist_fig.line(x[:-1], y, line_width=8,alpha=0.7,line_color="#D95B43",legend='Fit PDF')
    return x, y
    
# update histogram based on frequency, channel, bins on slider
def update_spct_hist(x, y, f=10, channel=0, numbins=20):
    plot_chan = int(channel.split(':')[0])
    plot_freq = np.where(lf.f_axis==f)[0][0]
    y, x = np.histogram(lf.spg[plot_chan,plot_freq,:], bins=numbins, density=True)
    # update histogram with data from frequency f
    hist_plot.data_source.data['left'] = x[:-1]
    hist_plot.data_source.data['right'] = x[1:]
    hist_plot.data_source.data['top'] = y    
    # update fitted
    rv = expon(scale=sp.stats.expon.fit(lf.spg[plot_chan,plot_freq,:],floc=0)[1])
    fit_plot.data_source.data['x'] = x
    fit_plot.data_source.data['y'] = rv.pdf(x)
    
    hist_fig.title.text = elec_def[elec_regions[plot_chan].astype(str)] + ': f=%.1fHz, p=%.4f '%(f,lf.ks_pvals[plot_chan,plot_freq])
    push_notebook()

# initializing plots
create_psd_plot(psd_plot, source, freq=select_freq)
create_scv_plot(scv_plot, source, freq=select_freq)
all_plots = gridplot([[psd_plot, scv_plot, hist_fig]], plot_width=300, plot_height=300)
create_hist(lf)

# what to do when channel selection changes
def selection_change(attr, old, new):
    chan = int(ticker.value)
    psd_vals = np.log(lf.psd[chan].T[1:])
    scv_vals = np.log(lf.scv[chan].T[1:])
    data = dict(freq_vals=freq_vals, psd_vals=psd_vals, scv_vals=scv_vals)
    # create a column data source for the plots to share
    source.data = data
    create_psd_plot(psd_plot=psd_plot, source=source, freq=select_freq)
    create_scv_plot(scv_plot=scv_plot, source=source, freq=select_freq)
    x,y = create_hist(lf=lf)
    update_spct_hist(x=x, y=y, f=select_freq, channel=chan)

# what to do when freq slider value changes
def freq_slider_callback(attr, old, new):
    new_freq = np.log(freq_slider.value)
    create_psd_plot(psd_plot=psd_plot, source=source, freq=new_freq)
    create_scv_plot(scv_plot=scv_plot, source=source, freq=new_freq)
    update_spct_hist(x=x, y=y, f=new_freq, channel=chan)

# what to do when freq slider value changes
def bin_slider_callback(attr, old, new):
    new_numbins = bin_slider.value
    update_spct_hist(x=x, y=y, f=new_freq, channel=chan, numbins=new_numbins)

ticker.on_change('value', selection_change)
freq_slider.on_change('value', freq_slider_callback)
bin_slider.on_change('value', freq_slider_callback)

# organize layout
widgets = row(ticker, freq_slider, bin_slider)
main_control = row(widgets)
layout = column(main_control, all_plots)

# show(widgetbox(select))
curdoc().add_root(layout)

show(layout)